In [1]:
import glob
import os
import struct
from nltk.tokenize import sent_tokenize

from tensorflow.core.example import example_pb2

In [2]:
file0 = '/home/francisco/GitHub/cnn-dailymail/finished_files/chunked/train_000.bin'

In [3]:
files = sorted(os.listdir('/home/francisco/GitHub/cnn-dailymail/finished_files/chunked/'))

load word vecs

Iterate through chunks of the 

In [4]:
def returnBytes(reader_obj):
    len_bytes = reader_obj.read(8)
    str_len = struct.unpack('q', len_bytes)[0]
    e_s = struct.unpack("%ds" % str_len, reader.read(str_len))
    es = e_s[0]
    c = example_pb2.Example.FromString(es)
    article  = str(c.features.feature['article'].bytes_list.value[0])
    abstract = str(c.features.feature['abstract'].bytes_list.value[0])
    ab = sent_tokenize(abstract)
    clean_article = sent_tokenize(article)
    clean_abstract = '. '.join([' '.join(s for s in x.split() if s.isalnum()) for x in ''.join(ab).replace("<s>","").split("</s>")]).strip()    
    return clean_abstract, clean_article

In [5]:
reader = open(file0, 'rb')

In [6]:
ab0, ar0 =  returnBytes(reader)
ab1, ar1 =  returnBytes(reader)

In [7]:
ab1

'harry potter star daniel radcliffe gets 20m fortune as he turns 18 monday. young actor says he has no plans to fritter his cash away. radcliffe earnings from first five potter films have been held in trust fund.'

In [8]:
import sys
import pandas as pd
sys.path.append('../src')
import data_io, params, SIF_embedding
import show_chunked

In [9]:
# input
wordfile = '/home/francisco/GitHub/SIF/data/glove.840B.300d.txt'   # word vector file, can be downloaded from GloVe website
weightfile = '/home/francisco/GitHub/SIF/auxiliary_data/enwiki_vocab_min200.txt' # each line is a word and its frequency
weightpara = 1e-3 # the parameter in the SIF weighting scheme, usually in the range [3e-5, 3e-3]
rmpc = 0
# rmpc = 1 # they usually set 1 ; number of principal components to remove in SIF weighting scheme

In [10]:
# load word vectors
(words, We) = data_io.getWordmap(wordfile)

In [11]:
# load word weights
word2weight = data_io.getWordWeight(weightfile, weightpara) # word2weight['str'] is the weight for the word 'str'
weight4ind = data_io.getWeight(words, word2weight) # weight4ind[i] is the weight for the i-th word

In [12]:
# set parameters
params = params.params()
params.rmpc = rmpc

In [13]:
sentences = ar0

In [14]:
# load sentences
x, m = data_io.sentences2idx(sentences, words) # x is the array of word indices, m is the binary mask indicating whether there is a word in that location
w = data_io.seq2weight(x, m, weight4ind) # get word weights

# get SIF embedding
embeddings = SIF_embedding.SIF_embedding(We, x, w, params) # embedding[i,:] is the embedding for sentence i

In [22]:
import numpy as np

In [27]:
# Sentence embeddings
sdf = pd.DataFrame(sentences, columns=['sentence'])
sdf['summary'] = ab0
sdf.ix[1:, 'summary']  = ''
emb = pd.DataFrame(embeddings, columns=['emb_%i' % x for x in range(embeddings.shape[1])])
sdf = pd.concat([sdf, emb], axis=1)

In [28]:
sdf.head()

,sentence,summary,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,...,emb_290,emb_291,emb_292,emb_293,emb_294,emb_295,emb_296,emb_297,emb_298,emb_299
0,"b""editor 's note : in our behind the scenes se...",mentally ill inmates in miami are housed on th...,-0.032912,0.042397,0.031326,0.148815,-0.005044,0.061765,-0.042820,0.020897,...,-0.155933,0.062875,-0.047902,-0.027060,-0.075693,-0.161862,-0.013952,-0.057433,-0.006306,-0.022029
1,"here , soledad o'brien takes users inside a ja...",,0.019653,0.105582,-0.032739,-0.062289,-0.007770,-0.096966,-0.034363,-0.116381,...,-0.045381,-0.101878,-0.103699,-0.002594,-0.056704,-0.028815,0.003903,-0.038061,-0.199252,0.018881
2,"an inmate housed on the `` forgotten floor , '...",,0.096693,0.013868,-0.015701,-0.101575,0.068017,-0.036213,-0.031631,-0.147130,...,-0.001189,-0.150899,-0.018053,0.050539,-0.083624,0.007458,0.082060,0.014343,-0.087338,-0.000591
3,"miami , florida -lrb- cnn -rrb- -- the ninth f...",,0.027993,0.086192,0.170021,0.144501,0.157027,0.029404,0.058522,-0.075552,...,0.101818,-0.118518,0.059976,0.031895,-0.104899,-0.052007,0.025132,0.088880,-0.156214,-0.061895
4,"here , inmates with the most severe mental ill...",,-0.007857,0.107740,-0.081650,-0.081380,0.005123,-0.116588,0.007774,-0.009557,...,-0.089699,-0.036341,-0.126280,-0.045463,-0.129258,0.005660,0.032350,-0.075145,-0.063582,-0.024497
